# H1 Intro
This exploration aims to:
- Compile a list of all planning applications for the past few years
- Compare top level analytics on these - approvals and rejections by county, year etc.
- Analyse application details to understand rejection reasons, timelines etc.

# H3 Imports & Variables
First, some necessary imports and globally applicable values

In [3]:
import pandas as pd
import plotly.graph_objects as go

import helper_functions as helper

filename = "IrishPlanningApplications.csv"

# H1 Baseline Data Gathering
Here we'll generate a CSV file of every planning application available through the [National Planning Applications](https://data.gov.ie/dataset/national-planning-applications) database. This will be our primary starting data source.

In [6]:
# Get all applications and return as a dataframe
df = helper.application_retriever()

# Save the DataFrame to a CSV file
df.to_csv(filename, index=False)
print("Saved records to ", filename)


About to begin fetching requests (this could take a few minutes)
Conversion failed for ExpiryDate with value 33556723200000. Keeping original value.
Conversion failed for ExpiryDate with value 33494515200000. Keeping original value.
Conversion failed for ExpiryDate with value 32474044800000. Keeping original value.
Conversion failed for ExpiryDate with value 32474044800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for GrantDate with value 96134860800000. Keeping original value.
Conversion failed for ExpiryDate with value 96292540800000. Keeping original value.
Conversion failed for ReceivedDate with value 38365315200000. Keeping or

In [5]:
# Load the dataset
df = pd.read_csv(filename)

# Display the first few rows of the DataFrame to verify it's loaded correctly
df.head()

/var/folders/1y/y7dky6b53zs0wszb6r1_6gmc0000gn/T/ipykernel_44617/1428047877.py:2: DtypeWarning: Columns (2,5,6,10,11,14,16,17,18,20,24,26,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


,OBJECTID,PlanningAuthority,ApplicationNumber,DevelopmentDescription,DevelopmentAddress,DevelopmentPostcode,ITMEasting,ITMNorthing,ApplicationStatus,ApplicationType,...,AppealSubmittedDate,FIRequestDate,FIRecDate,LinkAppDetails,OneOffKPI,ETL_DATE,SiteId,ORIG_FID,Latitude,Longitude
0,1.0,Carlow County Council,151,erection of a fully serviced two storey dormer...,"Clonogan , Clonegal , Co. Carlow",NaN,NaN,NaN,APPLICATION FINALISED,PERMISSION,...,NaN,2015-03-04 00:00:00,2015-03-11 00:00:00,http://www.eplanning.ie/CarlowCC/AppFileRefDet...,Yes,2023-04-27 15:13:22,NaN,1.0,6.928169e+06,-739925.245194
1,2.0,Carlow County Council,1510,"development at this site, Phase 2 Building, No...","Phase 2 Building , Northeast Elevation , Th...",NaN,NaN,NaN,APPLICATION FINALISED,PERMISSION,...,NaN,NaN,NaN,http://www.eplanning.ie/CarlowCC/AppFileRefDet...,No,2023-04-27 15:13:22,NaN,2.0,6.952309e+06,-770761.351635
2,3.0,Carlow County Council,15100,construct a domestic garage for domestic purpo...,"67 Park Gate , Shillelagh Road , Tullow",NaN,NaN,NaN,APPLICATION FINALISED,PERMISSION,...,NaN,NaN,NaN,http://www.eplanning.ie/CarlowCC/AppFileRefDet...,Yes,2023-04-27 15:13:22,NaN,3.0,6.947390e+06,-748265.781628
3,4.0,Carlow County Council,15101,alteration and extension to existing dwelling ...,"Knockevagh , Rathvilly , Co. Carlow",NaN,NaN,NaN,APPLICATION FINALISED,PERMISSION,...,NaN,NaN,NaN,http://www.eplanning.ie/CarlowCC/AppFileRefDet...,Yes,2023-04-27 15:13:22,NaN,4.0,6.961182e+06,-743359.200963
4,5.0,Carlow County Council,15102,construction of a dormer bungalow on an infill...,"Barrowlands , Leighlin Road , Graiguecullen",NaN,NaN,NaN,APPLICATION FINALISED,PERMISSION,...,NaN,2015-06-05 00:00:00,2015-09-01 00:00:00,http://www.eplanning.ie/CarlowCC/AppFileRefDet...,Yes,2023-04-27 15:13:22,NaN,5.0,6.952150e+06,-772320.581413


Simple test - analyze the distribution of planning application decisions (e.g., Granted, Refused) over the years

In [6]:
# Assuming df is existing DataFrame

# Filter the DataFrame for rows where PlanningAuthority is 'Galway County Council'
filtered_df = df[df['PlanningAuthority'].isin(['Galway County Council', 'Mayo County Council'])].copy()
# Drop potential missing values
filtered_df.dropna(subset=['DecisionDate', 'PlanningAuthority'], inplace=True)
# Convert decision dates to datetime format in the filtered DataFrame
filtered_df['DecisionDate'] = pd.to_datetime(filtered_df['DecisionDate'])

# Extract the year from the DecisionDate
filtered_df['DecisionYear'] = filtered_df['DecisionDate'].dt.year

# Group by 'DecisionYear', 'PlanningAuthority', and 'Decision' and count
decision_counts = filtered_df.groupby(['DecisionYear', 'PlanningAuthority', 'Decision']).size().reset_index(name='Counts')

# Prepare a figure
fig = go.Figure()

# Define unique categories
years = decision_counts['DecisionYear'].unique()
authorities = decision_counts['PlanningAuthority'].unique()
decisions = decision_counts['Decision'].unique()

# Create bars
for authority in authorities:
    for decision in decisions:
        df_filtered = decision_counts[(decision_counts['PlanningAuthority'] == authority) & (decision_counts['Decision'] == decision)]
        
        fig.add_trace(go.Bar(
            x=[f"{year}-{authority}" for year in df_filtered['DecisionYear']],
            y=df_filtered['Counts'],
            name=f"{authority} - {decision}",
            text=df_filtered['Counts'],
            textposition='auto',
        ))

# Customize appearance
fig.update_layout(
    barmode='stack',
    title='Planning Application Decisions by Year for Galway and Mayo County Councils',
    xaxis_title='Year - Planning Authority',
    yaxis_title='Number of Decisions',
    legend_title='Decision by Authority',
)

# Show the plot
fig.show()


Now, simplify our analysis into a simple table focusing on Galway County Council only first. this aims to replicate the test analysis in this doc - https://docs.google.com/spreadsheets/d/1dAIAc0Sm136LR-1PBHJ3R2-9v4FHaC-xnABoVQQ4PPc/edit#gid=993444100

In [30]:
galway_df = df[df['PlanningAuthority'] == 'Galway County Council'].copy()
galway_df['Received Year'] = pd.to_datetime(galway_df['ReceivedDate']).dt.year

results_df = pd.DataFrame(columns=['Received Year', 'Conditional Rate', 'Refusal Rate', 'Unconditional Rate'])

rows_list = []

for year in sorted(galway_df['Received Year'].unique()):
    year_data = galway_df[galway_df['Received Year'] == year]
    total_decisions = len(year_data)
    
    conditional_rate = ((year_data['Decision'].str.strip() == 'CONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    refusal_rate = ((year_data['Decision'].str.strip() == 'REFUSED').sum() / total_decisions * 100) if total_decisions > 0 else 0
    unconditional_rate = ((year_data['Decision'].str.strip() == 'UNCONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    
    rows_list.append({
        'Received Year': year,
        'Conditional Rate': round(conditional_rate, 2),
        'Refusal Rate': round(refusal_rate, 2),
        'Unconditional Rate': round(unconditional_rate, 2)
    })

results_df = pd.DataFrame(rows_list)
results_df['Received Year'] = results_df['Received Year'].astype(int)


# Ensure 'Received Year' column is of type int
results_df['Received Year'] = results_df['Received Year'].astype(int)

# Display the DataFrame
print(results_df)

   Received Year  Conditional Rate  Refusal Rate  Unconditional Rate
0           2015             68.61          3.94                9.01
1           2016             70.82          5.03                5.08
2           2017             76.52          5.77                2.86
3           2018             79.07          9.57                2.87
4           2019             73.05         13.40                1.97
5           2020             73.51         13.48                1.98
6           2021             71.45         17.39                2.35
7           2022             66.95         22.89                0.79
8           2023             62.35         14.61                0.06
9           2024              8.13          1.06                0.00


Same analysis, for Mayo this time

In [31]:
mayo_df = df[df['PlanningAuthority'] == 'Mayo County Council'].copy()
mayo_df['Received Year'] = pd.to_datetime(mayo_df['ReceivedDate']).dt.year

results_df = pd.DataFrame(columns=['Received Year', 'Conditional Rate', 'Refusal Rate', 'Unconditional Rate'])

rows_list = []

for year in sorted(mayo_df['Received Year'].unique()):
    year_data = mayo_df[mayo_df['Received Year'] == year]
    total_decisions = len(year_data)
    
    conditional_rate = ((year_data['Decision'].str.strip() == 'CONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    refusal_rate = ((year_data['Decision'].str.strip() == 'REFUSED').sum() / total_decisions * 100) if total_decisions > 0 else 0
    unconditional_rate = ((year_data['Decision'].str.strip() == 'UNCONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    
    rows_list.append({
        'Received Year': year,
        'Conditional Rate': round(conditional_rate, 2),
        'Refusal Rate': round(refusal_rate, 2),
        'Unconditional Rate': round(unconditional_rate, 2)
    })

results_df = pd.DataFrame(rows_list)
results_df['Received Year'] = results_df['Received Year'].astype(int)


# Ensure 'Received Year' column is of type int
results_df['Received Year'] = results_df['Received Year'].astype(int)

# Display the DataFrame
print(results_df)

   Received Year  Conditional Rate  Refusal Rate  Unconditional Rate
0           2015             88.48          4.41                 0.0
1           2016             88.34          4.88                 0.0
2           2017             87.49          5.27                 0.0
3           2018             87.59          5.21                 0.0
4           2019             83.49          8.15                 0.0
5           2020             82.76          7.43                 0.0
6           2021             84.35          8.13                 0.0
7           2022             79.66          6.10                 0.0
8           2023             69.49          4.28                 0.0
9           2024              6.76          0.68                 0.0
